In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
from astropy.io import fits
from scipy import optimize
import poppy
import gpipsfs
from scipy import signal
import os
import pytz
import time
import re

Kipac paths

In [ ]:
file1 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160229/aored_When_2016.2.29_5.33.19_poldm_phase.fits'
file2 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160227/aored_When_2016.2.27_0.2.8_poldm_phase.fits'

laptop paths

In [ ]:
file1 =  '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/aotelem/aored_When_2016.2.27_0.40.14_poldm_phase.fits'
file2 =  '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/aotelem/aored_When_2016.2.29_22.52.44_poldm_phase.fits'

Import images & remove static aberrations

In [ ]:
def import_fits(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')

    return phase

def remove_zernikes(phase):
    
    m1 = gpipsfs.GeminiPrimary().sample(npix=48)
    avg_phase = np.mean(phase*m1,axis=0)
    
    z_basis = poppy.zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = poppy.zernike.opd_expand_nonorthonormal(avg_phase, aperture=m1, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)
    c_phase = (phase - thin_lens[None,:,:])*m1
    c_phase[c_phase==0]=np.nan

    return c_phase

In [ ]:
phase = import_fits(file1)
c_phase = remove_zernikes(phase)
img = c_phase[0:20,:,:]
#img = img[~np.isnan(img)]

In [ ]:
img.shape

In [ ]:
%matplotlib notebook

plt.imshow(img)

In [ ]:
df = pd.DataFrame()

coord = [(i,j) for t in range(img.shape[0]) for i in range(img.shape[1]) for j in range(img.shape[2])]
tau = spatial.distance.cdist(coord,coord)


ph = [img[t,i,j] for t in range(img.shape[0]) for i in range(img.shape[1]) for j in range(img.shape[2])]
ph = np.reshape(np.array(ph),(len(ph),1))
print(ph)
phdiff = spatial.distance.cdist(ph,ph, metric='minkowski', p=2.)

df['tau'] = np.ravel(tau)
df['phdiff'] = np.ravel(phdiff)
df['bin'] = pd.cut(df['tau'],100)

In [ ]:
%matplotlib notebook

D = df.groupby('bin')['phdiff'].mean()

plt.plot(D.values)

In [ ]:
df = pd.DataFrame(index = np.unique(tau))

for i in range(602,612):
    if m[i]==np.nan:
        continue
    tau_i = np.unique(tau,axis=0)[i]
    for t in np.unique(tau_i):
        df.loc[t,'D'+str(i)] = np.nanmean((m[np.argwhere(tau_i==t)] - m[i])**2)


In [ ]:
%matplotlib notebook

plt.plot(df.index,df.mean(axis=1),'.')

Indexing 

In [ ]:
m = np.array([[0,1,2],[3,4,5],[6,7,8]])
m

In [ ]:
coord = [(i,j) for i in range(m.shape[0]) for j in range(m.shape[1])]
coord

In [ ]:
m = [[0,2,2],[3,0,0],[3,1,1]]

In [ ]:
tau = spatial.distance.cdist(coord,coord) 
#print(tau.size)
#print(tau[0])
tau_0 = np.unique(tau,axis=0)[0]

In [ ]:
m

In [ ]:
tau[1]

In [ ]:
np.unique(tau)

In [ ]:
m = np.ravel(m)
m

In [ ]:

df = pd.DataFrame(index = np.unique(tau),columns='D1')

for i in range(1):
    tau_i = np.unique(tau,axis=0)[i]
    for t in np.unique(tau_i):
        df.loc[t,'D'+str(i)] = np.nanmean((m[np.argwhere(tau_i==t)] - m[i])**2)
df

In [ ]:
df = pd.DataFrame(index = np.unique(tau),columns=['D0'])

df

In [ ]:
'D'+str(5)